<a href="https://colab.research.google.com/github/RifaldiAchmad/Maxim-App-Sentiment-Analysis/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import json

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
!git clone https://github.com/RifaldiAchmad/Maxim-App-Sentiment-Analysis.git

fatal: destination path 'Maxim-App-Sentiment-Analysis' already exists and is not an empty directory.


In [20]:
seed = 0
np.random.seed(seed)

In [21]:
# Membuat DataFrame dari hasil scraping
app_reviews_df = pd.read_excel('/content/Maxim-App-Sentiment-Analysis/maxim_training.xlsx')
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,3573ec27-522c-4048-83e2-0fe741d7fe71,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good job,5,0,3.16.6,2025-02-28 23:54:44,NaN,NaT,3.16.6
1,849a7551-1453-43a6-b318-26ce126761b4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pencariannya lama banget padahal banyak yg online,1,0,3.16.6,2025-02-28 23:45:49,NaN,NaT,3.16.6
2,55ecba1b-777c-43f2-9909-143fd7842b9c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,slalu pakai maxim,5,0,3.16.6,2025-02-28 23:00:33,NaN,NaT,3.16.6
3,e40c904e-2d38-457f-bd01-e2191b601e86,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ramah,5,0,3.16.6,2025-02-28 23:00:25,NaN,NaT,3.16.6
4,b5288f6d-ac87-49a3-809b-a00eb3bc60f5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,👍,5,0,3.16.6,2025-02-28 22:56:30,NaN,NaT,3.16.6


In [22]:
app_reviews_df.shape

(90380, 11)

In [23]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90380 entries, 0 to 90379
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              90380 non-null  object        
 1   userName              90380 non-null  object        
 2   userImage             90380 non-null  object        
 3   content               90380 non-null  object        
 4   score                 90380 non-null  int64         
 5   thumbsUpCount         90380 non-null  int64         
 6   reviewCreatedVersion  85172 non-null  object        
 7   at                    90380 non-null  datetime64[ns]
 8   replyContent          76286 non-null  object        
 9   repliedAt             76286 non-null  datetime64[ns]
 10  appVersion            85172 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 7.6+ MB


In [24]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = app_reviews_df.drop_duplicates()

In [25]:
# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
clean_df.shape

(90380, 11)

In [26]:
# Pastikan kolom 'at' bertipe datetime
clean_df['at'] = pd.to_datetime(clean_df['at'])

# Dapatkan tanggal maksimum dari data
latest_date = clean_df['at'].max()

# Tentukan batas waktu untuk 7 bulan terakhir (training)
training_start_date = latest_date - pd.DateOffset(months=7)

# Tentukan batas waktu untuk 3 bulan sebelumnya (inference)
inference_start_date = training_start_date - pd.DateOffset(months=5)

# Ambil data untuk training (7 bulan terakhir)
training_data = clean_df[clean_df['at'] >= training_start_date]

# Ambil data untuk inference (3 bulan sebelum training)
inference_data = clean_df[(clean_df['at'] >= inference_start_date) &
                                (clean_df['at'] < training_start_date)]

# Tampilkan dimensi dan contoh data
print("Training data shape:", training_data.shape)
print("Inference data shape:", inference_data.shape)

Training data shape: (53800, 11)
Inference data shape: (36580, 11)


## **Preprocessing Text**

In [27]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [28]:
def filteringText(text):
    # Daftar stopwords manual Bahasa Indonesia beserta variasi singkatannya
    stopwords_id = {
        "yang", "yg",
        "di",
        "ke",
        "dari", "dr",
        "dan",
        "atau",
        "itu",
        "ini",
        "untuk", "utk",
        "pada", "pd",
        "sebagai", "sebgai",
        "dengan", "dgn",
        "oleh", "olh",
        "karena", "krn",
        "jika", "jka", "jk",
        "tapi", "tpi", "tp",
        "namun",
        "jadi", "jdi", "jd",
        "saja", "aja", "aj",
        "adalah", "adl",
        "pula",
        "saat",
        "telah", "tlh",
        "akan",
        "bisa", "bs",
        "mereka",
        "kami",
        "kita",
        "anda",
        "tersebut", "tsb",
        "seperti", "sprti", "spt",
        "sekarang", "skrg",
        "masih",
        "agar",
        "tentang", "ttg",
        "selain",
        "antara",
        "kini",
        "demikian",
        "masing",
        "begitu",
        "sesuai",
        "hanya",
        "adapun",
        "lagi", "lg",
        "nya", "yaa", "loh", "kah", "woi", "woii", "woy",
        "klo",
        "sblm",
        "dg",
        "dpt",
        "wkwk",
        "saya",
        "buat"
    }
    # Hanya menghapus kata yang ada dalam daftar stopwords (perbandingan dengan huruf kecil)
    filtered = [word for word in text if word.lower() not in stopwords_id]
    return filtered

In [29]:
try:
    with open('/content/Maxim-App-Sentiment-Analysis/slang_words_id.txt', "r", encoding="utf-8") as f:
        slangwords_str = f.read().strip()  # Membaca file sebagai string

        if slangwords_str:
            slangwords = json.loads(slangwords_str)  # Konversi dari string ke dictionary
            print("Berhasil dikonversi ke dictionary!")
            print(list(slangwords.items())[:5])  # Menampilkan 5 contoh pertama
        else:
            print("File kosong!")
except FileNotFoundError:
    print("File tidak ditemukan! Periksa lokasi path-nya.")
except json.JSONDecodeError:
    print("Format JSON tidak valid! Periksa isi file.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Berhasil dikonversi ke dictionary!
[('@', 'di'), ('abis', 'habis'), ('ad', 'ada'), ('adlh', 'adalah'), ('afaik', 'as far as i know')]


In [30]:
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [31]:
training_data.dropna(subset=['content'], inplace=True)

In [32]:
training_data['text_clean'] = training_data['content'].apply(cleaningText) # Membersihkan teks
training_data['text_casefoldingText'] = training_data['text_clean'].apply(casefoldingText) # Mengubah huruf dalam teks menjadi huruf kecil
training_data['text_slangwords'] = training_data['text_casefoldingText'].apply(fix_slangwords) # Mengganti kata-kata slang dengan kata-kata standar
training_data['text_tokenizingText'] = training_data['text_slangwords'].apply(tokenizingText) # Memecah teks menjadi token (kata-kata)
training_data['text_stopword'] = training_data['text_tokenizingText'].apply(filteringText) # Menghapus kata-kata stop (kata-kata umum)
training_data['text_akhir'] = training_data['text_stopword'].apply(lambda tokens: ' '.join(tokens)) # Menggabungkan kata-kata yang telah distem menjadi kalimat

# **Feature Extraction & Labeling**

In [33]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [34]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    else:
         polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [35]:
results = training_data['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
training_data['polarity_score'] = results[0]
training_data['sentiment'] = results[1]
print(training_data['sentiment'].value_counts())

sentiment
positive    32655
negative    11264
neutral      9881
Name: count, dtype: int64


In [36]:
import os
import urllib.request
import zipfile
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [37]:
# Fungsi untuk mengunduh embedding GloVe jika belum tersedia
def download_glove(dimension=100):
    glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
    zip_path = "glove.6B.zip"

    # Jika file zip belum ada, unduh dari URL
    if not os.path.exists(zip_path):
        urllib.request.urlretrieve(glove_url, zip_path)

    glove_file = f"glove.6B.{dimension}d.txt"
    # Jika file GloVe spesifik dimensi belum ada, ekstrak dari zip
    if not os.path.exists(glove_file):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(glove_file)

    return glove_file

# Fungsi untuk memuat embedding GloVe dari file ke dalam dictionary
def load_glove_file(glove_file):
    embeddings_dict = {}
    try:
        # Buka file dan baca baris per baris
        with open(glove_file, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]  # Kata dari baris
                vector = np.array(values[1:], dtype='float32')  # Vektor numerik dari kata
                embeddings_dict[word] = vector  # Simpan ke dictionary
        return embeddings_dict
    except FileNotFoundError:
        return None

# Fungsi untuk membuat embedding dokumen dengan merata-ratakan vektor kata
def get_doc_embedding(doc, embeddings_dict, embedding_dim):
    words = doc.split()  # Pisahkan dokumen menjadi kata-kata
    doc_embedding = np.zeros(embedding_dim)  # Inisialisasi vektor dokumen dengan nol
    count = 0
    for word in words:
        if word in embeddings_dict:  # Jika kata ditemukan dalam embedding
            doc_embedding += embeddings_dict[word]  # Tambahkan vektor kata ke total
            count += 1
    if count > 0:
        doc_embedding /= count  # Rata-rata jika ada kata yang cocok
    return doc_embedding

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = training_data['text_akhir']
y = training_data['sentiment']

# Ekstraksi fitur menggunakan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Unduh dan muat embedding GloVe
embedding_dim = 100
glove_file = download_glove(embedding_dim)
embeddings = load_glove_file(glove_file)

# Membuat embedding dokumen menggunakan GloVe
print("Creating document embeddings...")
X_docs = []
for doc in X:
    X_docs.append(get_doc_embedding(doc, embeddings, embedding_dim))
X_glove = np.array(X_docs)

Creating document embeddings...


In [39]:
# Split data hasil Tf-Idf
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [40]:
# Split data hasil GloVe embeddings
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_glove, y, test_size=0.2, random_state=42)

# **Modeling**

### EKSPERIMEN 1:

Pelatihan : SVM

Ekstraksi Fitur : TF-IDF

Pembagian Data : 80/20

In [41]:
# Pelatihan SVM dengan kernel rbf
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)

# Evaluasi pada data pelatihan
y_pred_train = svm.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train, average='weighted')
train_recall = recall_score(y_train, y_pred_train, average='weighted')
train_f1 = f1_score(y_train, y_pred_train, average='weighted')

# Evaluasi pada data pengujian
y_pred_test = svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data = {
    'Accuracy': [train_accuracy, test_accuracy],
    'Precision': [train_precision, test_precision],
    'Recall': [train_recall, test_recall],
    'F1 Score': [train_f1, test_f1]
}

metrics_df = pd.DataFrame(metrics_data, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik SVM:")
metrics_df


Tabel Evaluasi Metrik SVM:


,Accuracy,Precision,Recall,F1 Score
Train,0.947421,0.948348,0.947421,0.947755
Test,0.932249,0.933246,0.932249,0.932608


### EKSPERIMEN 2:

Pelatihan : Random Forest

Ekstraksi Fitur : TF-IDF

Pembagian Data : 80/30

In [42]:
# Pelatihan Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluasi pada data pelatihan
y_pred_train_1 = rf.predict(X_train)
train_accuracy_1 = accuracy_score(y_train, y_pred_train_1)
train_precision_1 = precision_score(y_train, y_pred_train_1, average='weighted')
train_recall_1 = recall_score(y_train, y_pred_train_1, average='weighted')
train_f1_1 = f1_score(y_train, y_pred_train_1, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_1 = rf.predict(X_test)
test_accuracy_1 = accuracy_score(y_test, y_pred_test_1)
test_precision_1 = precision_score(y_test, y_pred_test_1, average='weighted')
test_recall_1 = recall_score(y_test, y_pred_test_1, average='weighted')
test_f1_1 = f1_score(y_test, y_pred_test_1, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_1 = {
    'Accuracy': [train_accuracy_1, test_accuracy_1],
    'Precision': [train_precision_1, test_precision_1],
    'Recall': [train_recall_1, test_recall_1],
    'F1 Score': [train_f1_1, test_f1_1]
}

metrics_df_1 = pd.DataFrame(metrics_data_1, index=['Train', 'Test'])
print("\nTabel Evalusi Metrik Random Forest:")
metrics_df_1


Tabel Evalusi Metrik Random Forest:


,Accuracy,Precision,Recall,F1 Score
Train,0.967240,0.969051,0.967240,0.967654
Test,0.926022,0.927203,0.926022,0.926395


### EKSPERIMEN 3:

Pelatihan : SVM

Ekstraksi Fitur : GloVe

Pembagian Data : 80/20

In [43]:
# Pelatihan SVM dengan kernel rbf
svm_glove = SVC(kernel='rbf')
svm_glove.fit(X_train1, y_train1)

# Evaluasi pada data pelatihan
y_pred_train_2 = svm_glove.predict(X_train1)
train_accuracy_2 = accuracy_score(y_train1, y_pred_train_2)
train_precision_2 = precision_score(y_train1, y_pred_train_2, average='weighted')
train_recall_2 = recall_score(y_train1, y_pred_train_2, average='weighted')
train_f1_2 = f1_score(y_train1, y_pred_train_2, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_2 = svm_glove.predict(X_test1)
test_accuracy_2 = accuracy_score(y_test1, y_pred_test_2)
test_precision_2 = precision_score(y_test1, y_pred_test_2, average='weighted')
test_recall_2 = recall_score(y_test1, y_pred_test_2, average='weighted')
test_f1_2 = f1_score(y_test1, y_pred_test_2, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_2 = {
    'Accuracy': [train_accuracy_2, test_accuracy_2],
    'Precision': [train_precision_2, test_precision_2],
    'Recall': [train_recall_2, test_recall_2],
    'F1 Score': [train_f1_2, test_f1_2]
}

metrics_df_2 = pd.DataFrame(metrics_data_2, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik SVM:")
metrics_df_2


Tabel Evaluasi Metrik SVM:


,Accuracy,Precision,Recall,F1 Score
Train,0.794424,0.794344,0.794424,0.776974
Test,0.789312,0.787200,0.789312,0.772806


### EKSPERIMEN 4:

Pelatihan : Random Forest

Ekstraksi Fitur : GloVe

Pembagian Data : 80/20

In [44]:
# Pelatihan Random Forest
rf_glove = RandomForestClassifier(random_state=42)
rf_glove.fit(X_train1, y_train1)

# Evaluasi pada data pelatihan
y_pred_train_3 = rf_glove.predict(X_train1)
train_accuracy_3 = accuracy_score(y_train1, y_pred_train_3)
train_precision_3 = precision_score(y_train1, y_pred_train_3, average='weighted')
train_recall_3 = recall_score(y_train1, y_pred_train_3, average='weighted')
train_f1_3 = f1_score(y_train1, y_pred_train_3, average='weighted')

# Evaluasi pada data pengujian
y_pred_test_3 = rf_glove.predict(X_test1)
test_accuracy_3 = accuracy_score(y_test1, y_pred_test_3)
test_precision_3 = precision_score(y_test1, y_pred_test_3, average='weighted')
test_recall_3 = recall_score(y_test1, y_pred_test_3, average='weighted')
test_f1_3 = f1_score(y_test1, y_pred_test_3, average='weighted')

# Menyimpan hasil evaluasi ke dalam DataFrame
metrics_data_3 = {
    'Accuracy': [train_accuracy_3, test_accuracy_3],
    'Precision': [train_precision_3, test_precision_3],
    'Recall': [train_recall_3, test_recall_3],
    'F1 Score': [train_f1_3, test_f1_3]
}

metrics_df_3 = pd.DataFrame(metrics_data_3, index=['Train', 'Test'])
print("\nTabel Evaluasi Metrik Random Forest:")
metrics_df_3


Tabel Evaluasi Metrik Random Forest:


,Accuracy,Precision,Recall,F1 Score
Train,0.846561,0.845351,0.846561,0.835256
Test,0.783829,0.780865,0.783829,0.768035


# **Inferensi (Random Forest, TF-IDF, Train 96.7% dan Test 92.6%)**

In [45]:
inference_data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
53800,f5ca2669-21a4-427d-adfe-2ffd5493ddc0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,cukup memuaskan,5,0,3.16.2,2024-07-28 23:54:27,Review kamu bikin kita makin semangat! 💪 Makas...,2024-07-29 12:03:05,3.16.2
53801,9bd1b787-b578-4bda-a1cb-b7a7d6a0f50e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"membantu sekali untuk perjalanan, driver juga ...",5,0,3.16.2,2024-07-28 23:52:59,Review kamu bikin kita makin semangat! 💪 Makas...,2024-07-29 12:03:04,3.16.2
53802,a381b4a0-1781-40b7-8e98-2740f67952f8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Tidak ada menu ""uang"" jadi tidak bisa top up a...",1,0,3.16.2,2024-07-28 23:46:07,Terima kasih atas umpan balik Anda 😢 Kami akan...,2024-07-29 12:46:45,3.16.2
53803,15715bac-0ee3-4e05-bf0d-ff7d7e4c3652,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good job,5,0,3.15.16,2024-07-28 23:29:36,Makasih banyak atas review-nya! 😊 Seneng bange...,2024-07-29 12:03:42,3.15.16
53804,45be057a-b6ea-4d60-b78e-5e26dcab0230,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"the best bwt maxim,harga bener"" terjangkau apa...",5,0,NaN,2024-07-28 23:28:53,Makasih banyak atas review-nya! 😊 Seneng bange...,2024-07-29 12:03:40,NaN


Menggunakan model

In [46]:
# Transformasi teks yang sudah dipreproses menggunakan vectorizer TF-IDF yang telah dilatih
X_pred = tfidf.transform(inference_data['content'])

# Lakukan prediksi
inference_data['sentiment'] = rf.predict(X_pred)
inference_data

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment
53800,f5ca2669-21a4-427d-adfe-2ffd5493ddc0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,cukup memuaskan,5,0,3.16.2,2024-07-28 23:54:27,Review kamu bikin kita makin semangat! 💪 Makas...,2024-07-29 12:03:05,3.16.2,negative
53801,9bd1b787-b578-4bda-a1cb-b7a7d6a0f50e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"membantu sekali untuk perjalanan, driver juga ...",5,0,3.16.2,2024-07-28 23:52:59,Review kamu bikin kita makin semangat! 💪 Makas...,2024-07-29 12:03:04,3.16.2,positive
53802,a381b4a0-1781-40b7-8e98-2740f67952f8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Tidak ada menu ""uang"" jadi tidak bisa top up a...",1,0,3.16.2,2024-07-28 23:46:07,Terima kasih atas umpan balik Anda 😢 Kami akan...,2024-07-29 12:46:45,3.16.2,negative
53803,15715bac-0ee3-4e05-bf0d-ff7d7e4c3652,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good job,5,0,3.15.16,2024-07-28 23:29:36,Makasih banyak atas review-nya! 😊 Seneng bange...,2024-07-29 12:03:42,3.15.16,positive
53804,45be057a-b6ea-4d60-b78e-5e26dcab0230,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"the best bwt maxim,harga bener"" terjangkau apa...",5,0,NaN,2024-07-28 23:28:53,Makasih banyak atas review-nya! 😊 Seneng bange...,2024-07-29 12:03:40,NaN,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
90375,3ab2b0f8-cea8-4007-b2e8-9ad39c0de6ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,puas dengan layanan,5,0,3.15.16,2024-03-01 00:25:44,Halo. Terima kasih atas ulasan positifnya. Kam...,2024-03-01 02:51:32,3.15.16,positive
90376,348d1eed-8d80-40af-8354-8cd7227aa0ab,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus sekali,5,0,3.15.15,2024-03-01 00:21:39,Halo. Terima kasih atas ulasan positifnya. Kam...,2024-03-01 02:51:32,3.15.15,negative
90377,a758ae97-45cd-4713-a65f-b9ecbec21d26,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"good,",5,0,3.15.15,2024-03-01 00:18:34,Halo. Terima kasih atas ulasan positifnya. Kam...,2024-03-01 02:49:42,3.15.15,positive
90378,861c4580-a07f-4a05-a80b-d84cd8094edd,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,drivernya buaik babget,5,0,3.15.16,2024-03-01 00:07:35,Halo. Terima kasih atas ulasan positifnya. Kam...,2024-03-01 02:49:39,3.15.16,neutral


In [47]:
inference_data.sentiment.value_counts()

,count
sentiment,
positive,21923
neutral,7449
negative,7208
